In [1]:
# Exercises 2, 3

In [1]:
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd

import classifier as cl
import dtransform as dt
import stats
import subgraphs as sg

In [2]:
alz_data = pd.read_csv("datasets/AlzheimersDiseaseTrain.csv")
alz_data.head()

,CLASS,AD,AD.1,AD.2,AD.3,AD.4,AD.5,AD.6,AD.7,AD.8,...,NDC.30,NDC.31,NDC.32,NDC.33,NDC.34,NDC.35,NDC.36,NDC.37,NDC.38,NDC.39
0,ANG_1,4.520104,4.430224,4.244593,2.906250,3.106781,1.966611,3.670303,3.365818,4.404305,...,2.687549,3.918413,3.904447,3.173363,3.819013,2.087533,3.460492,3.070150,3.561487,2.153643
1,BDNF_1,2.692648,0.945860,2.916182,-0.262353,0.084827,-0.024774,0.691213,-0.306191,0.397225,...,1.255353,-0.465786,0.099672,1.109036,1.962476,1.739724,2.046212,1.473774,1.710057,0.868764
2,BLC_1,0.788355,0.243013,1.029638,-0.651429,-0.530144,-0.466503,-0.207471,-0.598578,-0.247810,...,-1.047142,-0.752896,-0.646474,-0.577310,0.163524,-0.063873,1.252525,0.851452,0.189141,-0.184288
3,BMP-4_1,1.142045,0.407405,1.966551,-0.032887,0.239389,-0.309028,1.110751,0.516942,1.267004,...,0.713331,-0.160065,0.647319,-0.091960,1.239154,2.019634,2.652180,2.394194,1.713033,1.956316
4,BMP-6_1,-0.376951,0.480273,2.397770,-0.455636,-0.567477,-0.444746,0.603011,-0.210501,0.447816,...,-0.288518,-0.386555,-0.378309,-0.386273,0.467163,1.440439,1.933714,0.256852,1.135844,1.599819


In [3]:
# Feature (protein) labels
protein_labels = []
for label in alz_data.iloc[:, 0]:
    protein_labels.append(str(label))
    
# Instance labels
inst_labels = []
for label in alz_data.columns:
    inst_labels.append(str(label))
inst_labels.remove('CLASS')
print(inst_labels)

['AD', 'AD.1', 'AD.2', 'AD.3', 'AD.4', 'AD.5', 'AD.6', 'AD.7', 'AD.8', 'AD.9', 'AD.10', 'AD.11', 'AD.12', 'AD.13', 'AD.14', 'AD.15', 'AD.16', 'AD.17', 'AD.18', 'AD.19', 'AD.20', 'AD.21', 'AD.22', 'AD.23', 'AD.24', 'AD.25', 'AD.26', 'AD.27', 'AD.28', 'AD.29', 'AD.30', 'AD.31', 'AD.32', 'AD.33', 'AD.34', 'AD.35', 'AD.36', 'AD.37', 'AD.38', 'AD.39', 'AD.40', 'AD.41', 'AD.42', 'NDC', 'NDC.1', 'NDC.2', 'NDC.3', 'NDC.4', 'NDC.5', 'NDC.6', 'NDC.7', 'NDC.8', 'NDC.9', 'NDC.10', 'NDC.11', 'NDC.12', 'NDC.13', 'NDC.14', 'NDC.15', 'NDC.16', 'NDC.17', 'NDC.18', 'NDC.19', 'NDC.20', 'NDC.21', 'NDC.22', 'NDC.23', 'NDC.24', 'NDC.25', 'NDC.26', 'NDC.27', 'NDC.28', 'NDC.29', 'NDC.30', 'NDC.31', 'NDC.32', 'NDC.33', 'NDC.34', 'NDC.35', 'NDC.36', 'NDC.37', 'NDC.38', 'NDC.39']


In [4]:
alz_data = alz_data.iloc[:, 1:]
alz_data.head()

,AD,AD.1,AD.2,AD.3,AD.4,AD.5,AD.6,AD.7,AD.8,AD.9,...,NDC.30,NDC.31,NDC.32,NDC.33,NDC.34,NDC.35,NDC.36,NDC.37,NDC.38,NDC.39
0,4.520104,4.430224,4.244593,2.906250,3.106781,1.966611,3.670303,3.365818,4.404305,4.521486,...,2.687549,3.918413,3.904447,3.173363,3.819013,2.087533,3.460492,3.070150,3.561487,2.153643
1,2.692648,0.945860,2.916182,-0.262353,0.084827,-0.024774,0.691213,-0.306191,0.397225,0.276861,...,1.255353,-0.465786,0.099672,1.109036,1.962476,1.739724,2.046212,1.473774,1.710057,0.868764
2,0.788355,0.243013,1.029638,-0.651429,-0.530144,-0.466503,-0.207471,-0.598578,-0.247810,-0.569392,...,-1.047142,-0.752896,-0.646474,-0.577310,0.163524,-0.063873,1.252525,0.851452,0.189141,-0.184288
3,1.142045,0.407405,1.966551,-0.032887,0.239389,-0.309028,1.110751,0.516942,1.267004,0.140410,...,0.713331,-0.160065,0.647319,-0.091960,1.239154,2.019634,2.652180,2.394194,1.713033,1.956316
4,-0.376951,0.480273,2.397770,-0.455636,-0.567477,-0.444746,0.603011,-0.210501,0.447816,-0.509188,...,-0.288518,-0.386555,-0.378309,-0.386273,0.467163,1.440439,1.933714,0.256852,1.135844,1.599819


In [5]:
def eucl_dist(x, y):
    sum = 0
    for i in range(len(x)):
        sum += (x[i] - y[i])**2
        
    return math.sqrt(sum)

In [7]:
# Attributes are rows, instances are columns
nattr, ninst = alz_data.shape
D = np.zeros((ninst, ninst))

for i in range(ninst):
    i_data = alz_data.iloc[:, i].values
    for j in range(i, ninst):
        j_data = alz_data.iloc[:, j].values
        
        dist = eucl_dist(i_data, j_data)
        D[i][j] = D[j][i] = dist
if os.path.exists("Output"):
    print("Path exists")
else:
    print("Path not found, creating directory")
    os.mkdir("Output")
    
np.savetxt("Output/AlzheimersInstanceDistMatrix.csv", D, fmt="%.2f", delimiter=",")

Path exists


In [8]:
D_attr = np.zeros((nattr, nattr))

for i in range(nattr):
    i_data = alz_data.iloc[i, :].values
    for j in range(i, nattr):
        j_data = alz_data.iloc[j, :].values
        
        dist = eucl_dist(i_data, j_data)
        D_attr[i][j] = D_attr[j][i] = dist

if os.path.exists("Output"):
    print("Path exists")
else:
    print("Path not found, creating directory")
    os.mkdir("Output")
    
np.savetxt("Output/AlzheimersAttributeDistMatrix.csv", D_attr, fmt="%.2f", delimiter=",")

Path exists


In [9]:
# Generate sets of vertices
V = set(n for n in range(len(D)))
V_attr = set(n for n in range(len(D_attr)))

In [10]:
_ = sg.rngraph(V, D, "Output/alz-inst-rng.gml", inst_labels)
_ = sg.gg_graph(V, D, "Output/alz-inst-gg.gml", inst_labels)

In [11]:
_ = sg.rngraph(V_attr, D_attr, "Output/alz-attr-rng.gml", protein_labels)
_ = sg.gg_graph(V_attr, D_attr, "Output/alz-attr-gg.gml", protein_labels)

In [12]:
# Exercise 3

In [6]:
# Retrieve classes from dataframe and clean pandas numerical info
import re

raw_classes = alz_data.iloc[0, :].index
classes = []
for item in raw_classes:
    classes.append(re.sub('.\d+','', item))
    
print(classes)

['AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC', 'NDC']


In [7]:
import importlib
importlib.reload(dt)

<module 'dtransform' from '/home/conor/Documents/COMP3340/Assignments/P2/dtransform.py'>

In [8]:
# Create a discretised version of the dataset
alz_data_disc = alz_data.copy()
alz_data_disc.columns = [i for i in range(len(alz_data_disc.columns))]
alz_splits = dt.discretise_dataset(alz_data_disc, classes, attr_axis=0)
alz_data_disc.to_csv("Output/AlzheimersDiscrete.csv")

Total features =  120


In [9]:
alz_data_disc.head()

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Test discretising a discrete dataset
dt.discretise_dataset(alz_data_disc, classes, attr_axis=0)
alz_data_disc.head()

Total features =  120


,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
alz_data_disc.to_csv("temp.csv")

In [49]:
class_labels = dt.map_classes(classes, 'AD')
print(class_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [50]:
alz_data_reduced = pd.DataFrame()

keep = []
for i, row in alz_data_disc.iterrows():
    delete = True
    for val in row:
        if val != 0:
            delete = False
            
    if not delete:
        keep.append(i)
        
alz_data_reduced = alz_data_disc.filter(keep, axis=0)
alz_data_reduced.to_csv("Output/AlzheimersDiscreteReduced.csv")

In [51]:
for i, row in alz_data_reduced.iterrows():
    print(protein_labels[i])

IL-1a_1
IL-6_1
MCP-3_1
RANTES_1
TNF-a_1
GCSF_1
IL-11_1


In [31]:
freq_tables = cl.naive_bayes(alz_data_reduced, class_labels, attr_axis=0)

# Test on train dataset
classifications = cl.nb_classify(freq_tables, alz_data_reduced, class_labels, inst_axis=1)

# Evaluate classifier

# Produce confusion matrix
# TP FP
# FN TN
cm = cl.cmatrix(classifications, class_labels)

print(cm)
print(f'Sensitivity: {stats.sensitivity(cm):.2f}')
print(f'Specificity: {stats.specificity(cm): .2f}')
print(f'Accuracy: {stats.accuracy(cm): .2f}')
print(f'f1-score: {stats.f1(cm): .2f}')
print(f"Matthew's Correlation Coefficient: {stats.mcc(cm): .2f}")
print(f"Youden's J: {stats.youdenJ(cm): .2f}")

[[39.  3.]
 [ 4. 37.]]
Sensitivity: 0.91
Specificity:  0.93
Accuracy:  0.92
f1-score:  0.92
Matthew's Correlation Coefficient:  0.83
Youden's J:  0.83


In [32]:
# Prepare test dataset
alz_test = pd.read_csv("datasets/AlzheimersDiseaseTest.csv")
alz_test = alz_test.iloc[:, 1:]  # Remove feature labels column

# Retrieve classes
raw_test_classes = alz_test.iloc[0, :].index
test_classes = []
for item in raw_test_classes:
    # Turn into a binary classification problem
    string = re.sub('.\d+','', item)
    string = re.sub('NDC','notAD',string)
    string = re.sub('OD','notAD',string)
    test_classes.append(string)
        
for index in range(len(alz_test.index)):
    cur_data = alz_test.iloc[index, :]
    dt.apply_splits(cur_data, alz_splits[index])

    
alz_test = alz_test.filter(keep, axis=0)

In [33]:
# Classify test dataset
test_labels = dt.map_classes(test_classes, 'AD')
classifications = cl.nb_classify(freq_tables, alz_test, test_labels, inst_axis=1)
cm = cl.cmatrix(classifications, test_labels)

print(cm)
print(f'Sensitivity: {stats.sensitivity(cm):.2f}')
print(f'Specificity: {stats.specificity(cm): .2f}')
print(f'Accuracy: {stats.accuracy(cm): .2f}')
print(f'f1-score: {stats.f1(cm): .2f}')
print(f"Matthew's Correlation Coefficient: {stats.mcc(cm): .2f}")
print(f"Youden's J: {stats.youdenJ(cm): .2f}")

[[31.  7.]
 [11. 43.]]
Sensitivity: 0.74
Specificity:  0.86
Accuracy:  0.80
f1-score:  0.78
Matthew's Correlation Coefficient:  0.61
Youden's J:  0.60


In [34]:
# Prepare MCI-labelled samples dataset
alz_mci = pd.read_csv("datasets/AlzheimersDiseaseTestMCI.csv")
alz_mci = alz_mci.iloc[:, 1:]  # Remove feature labels column
alz_mci = alz_mci.iloc[1:, :]  # Remove follow up diagnosis row

# Adjust indices after removing a row - without this, wrong rows are selected when reducing dataset
alz_mci = alz_mci.reindex([i for i in range(120)], method='backfill')

# Retrieve classes
raw_mci_classes = alz_mci.iloc[0, :].index
mci_classes = []
for item in raw_mci_classes:
    # Turn into a binary classification problem
    string = re.sub('.\d+','', item)
    string = re.sub('MCI-MCI','notAD',string)
    string = re.sub('MCI-OD','notAD',string)
    mci_classes.append(string)
        
for index in range(len(alz_mci.index)):
    cur_data = alz_mci.iloc[index, :]
    dt.apply_splits(cur_data, alz_splits[index])

alz_mci = alz_mci.filter(keep, axis=0)

In [35]:
# Classify MCI-labelled samples dataset
mci_labels = dt.map_classes(mci_classes, 'MCI-AD')
classifications = cl.nb_classify(freq_tables, alz_mci, mci_labels, inst_axis=1)
cm = cl.cmatrix(classifications, mci_labels)

print(cm)
print(f'Sensitivity: {stats.sensitivity(cm):.2f}')
print(f'Specificity: {stats.specificity(cm): .2f}')
print(f'Accuracy: {stats.accuracy(cm): .2f}')
print(f'f1-score: {stats.f1(cm): .2f}')
print(f"Matthew's Correlation Coefficient: {stats.mcc(cm): .2f}")
print(f"Youden's J: {stats.youdenJ(cm): .2f}")

[[19. 18.]
 [ 3.  7.]]
Sensitivity: 0.86
Specificity:  0.28
Accuracy:  0.55
f1-score:  0.64
Matthew's Correlation Coefficient:  0.18
Youden's J:  0.14
